# Google Play Scraper for CI and Sentiment Analysis

In [1]:
#!pip install -qq google-play-scraper

In [2]:
import json
import pandas as pd
from tqdm import tqdm

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

## Create data storage

In [3]:
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
path = "Downloads"

## Targeting the Company

In [4]:
app_packages = [
                'ovo.id',
                'id.dana'
]

In [5]:
app_infos = []

for application in tqdm(app_packages):
  info = app(application, lang='id', country='id')
  del info['comments']
  app_infos.append(info)

100%|██████████| 2/2 [00:05<00:00,  2.92s/it]


In [6]:
def print_json(json_object):
  json_str = json.dumps(
      json_object,
      indent=2,
      sort_keys= True,
      default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [7]:
print_json(app_infos[0])

{
  "adSupported": null,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 dan yang lebih tinggi",
  "appId": "ovo.id",
  "containsAds": false,
  "contentRating": "Rating 3+",
  "contentRatingDescription": null,
  "currency": "IDR",
  "description": "Dari jajan-jajan sampai makan besar, dari tagihan bulanan sampai belanja dadakan, dari online shopping sampai warung pinggir jalan \u2013 kapan pun dan di mana pun, semuanya bisa dibayar cukup #pakeOVOaja!\r\n\r\nYup bener, sekarang semuanya cukup pake OVO aja. Gak perlu repot sama uang cash, transaksi lebih mudah, cepat, dan aman pake OVO.\r\n\r\n\r\nMau tau OVO bisa ngapain aja?\r\n\r\n> Bisa Bayar Ini-Itu\r\nPakai OVO untuk transaksimu di 500.000+ merchant rekanan, beli pulsa, bayar tagihan bulanan, parkir, semua layanan Grab, dan semua pembelian di Tokopedia.\r\n\r\n> Hemat Transaksimu di Mana-Mana dengan OVO Points\r\n1 OVO Point = Rp1. Dapatkan OVO Points saat kamu bertransaksi di merchant rekanan, dan gunakan untuk menghemat t

## Data Extraction

In [8]:
def format_title(title):
  sep_index = title.find(':') if title.find(':') != -1 else title.find('-')
  if sep_index != -1:
    title = title[:sep_index]
  return title[:10]

In [9]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv(path +'apps.csv', index=None, header=True)

In [10]:
app_reviews = []

for ap in tqdm(app_packages):
 for score in list(range(1,6)):
    for sort_order in (Sort.MOST_RELEVANT, Sort.NEWEST):
      rvs, _ = reviews(
          ap,
          lang='id',
          country='id',
          sort = sort_order,
          count = 1200 if score == 3 else 600,
          filter_score_with = score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)


100%|██████████| 2/2 [00:49<00:00, 24.90s/it]


In [11]:
print_json(app_reviews[0])

{
  "appId": "ovo.id",
  "at": "2021-09-08 13:19:16",
  "content": "Hay OVO..Gak usah banyak gaya ingin kerja sama dengan adira..nampung prakerja aja sudah kewalahan server kamu. Perbarui dulu sistem nya,upgrade server nya baru tingkatkan pelayanan. Ini saya pengguna lama ovo premier,mau login susah kode otp gak dikirim kirim. Hubungi CS abis pulsa puluhan ribu gak ada hasil. Gimana mau transaksi,sudah 3 hari gak bisa login. Saldo saya jadi gak berguna klo begitu. Pantas rating bintang satu semua. TERTINGGAL JAUH oleh DANA,GOPAY.",
  "repliedAt": "2021-09-08 13:36:41",
  "replyContent": "Hai Kak Vian, mohon maaf atas ketidaknyamanannya. Apabila ada keluhan/pertanyaan mohon menghubungi tim kami melalui email di cs@ovo.id atau call center kami di 1500-696. Terima kasih. ^YP",
  "reviewCreatedVersion": "3.42.0",
  "reviewId": "gp:AOqpTOFfVB-ecjLtidhryYJFhh4T0Z1KDzcs-WaGqb8A_P7xSuFeK--2adFSbRETBZvFl_fZQnVYPTlh278_sPM",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 82,
  

In [12]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv(path+'ggplay.csv',index=None, header=True)

In [16]:
# Read Result
df = pd.read_csv('ggplay.csv')
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOFfVB-ecjLtidhryYJFhh4T0Z1KDzcs-WaGqb8...,Vian Alfiansyah,https://play-lh.googleusercontent.com/a/AATXAJ...,Hay OVO..Gak usah banyak gaya ingin kerja sama...,1,82,3.42.0,2021-09-08 13:19:16,"Hai Kak Vian, mohon maaf atas ketidaknyamanann...",2021-09-08 13:36:41,most_relevant,ovo.id
1,gp:AOqpTOFgnPyjzAh6XXZKgBigKp2IIfzoOKSlxiN-Igw...,Afrizal Santoso,https://play-lh.googleusercontent.com/a-/AOh14...,"Sungguh bikin kesal, saya dari kemarin beli pa...",1,4,3.42.0,2021-09-09 11:23:41,"Hai Kak Afrizal, mohon maaf atas ketidaknyaman...",2021-09-09 11:29:46,most_relevant,ovo.id
2,gp:AOqpTOGuj97RQy6mgaT1SV-8s5lxm2jK9ZiAHS1fAgE...,Ekki Budiansyah Putri utami,https://play-lh.googleusercontent.com/a-/AOh14...,Ga guna banget nih aplikasi. Sekarang susah ba...,1,1,3.42.0,2021-09-09 19:20:05,"Hai Kak Ekki, mohon maaf atas ketidaknyamanann...",2021-09-09 19:25:21,most_relevant,ovo.id
3,gp:AOqpTOHdA2lJTQACd2sEqh73r-5ow4_40SZaeKfABDM...,Syofriwan Hadi,https://play-lh.googleusercontent.com/a-/AOh14...,banyak respon negatif dari user 😂😂😂 *top up bi...,1,28,3.42.0,2021-09-07 07:58:40,"Hai Kak Syofriwan, mohon maaf atas ketidaknyam...",2021-09-07 08:09:21,most_relevant,ovo.id
4,gp:AOqpTOGoM9YMrnunBDtgTyY1HuFqMiCCb5euqKFhLhG...,Imas Siti masitoh,https://play-lh.googleusercontent.com/a-/AOh14...,Ini gimana yah ko insentif saya belum cair? Pd...,1,25,3.42.0,2021-09-07 06:07:20,"Hai Kak, mohon maaf atas ketidaknyamanannya, a...",2021-09-07 06:18:09,most_relevant,ovo.id
...,...,...,...,...,...,...,...,...,...,...,...,...
14395,gp:AOqpTOGe4EzVlFtrhYK_3Kq1Ac7x5B9uC65DPKylFou...,Ayah Azizah,https://play-lh.googleusercontent.com/a/AATXAJ...,"sangat membantu,maaf kak aplikasi d dana saya ...",5,0,2.2.0,2021-09-10 19:49:43,"Hi Kak, maaf buat kecewa. Terkait keluhan yg d...",2021-09-10 21:58:40,newest,id.dana
14396,gp:AOqpTOEU4Hl4hFR7dPhJqmXWVaK665pV_eL0XUJtYOF...,Mulyani Mulyani,https://play-lh.googleusercontent.com/a/AATXAJ...,ok,5,0,2.1.1,2021-09-10 19:47:52,"Hi Kak, terima kasih ya sudah mempercayakan DA...",2021-09-10 21:31:38,newest,id.dana
14397,gp:AOqpTOEBvDTvuzHXNKxX69jp3YaetjktIxR7nQNRcYz...,Desi.M Chanel,https://play-lh.googleusercontent.com/a/AATXAJ...,baik,5,0,2.2.0,2021-09-10 19:47:26,"Hi Kak, terima kasih ya sudah mempercayakan DA...",2021-09-10 21:31:41,newest,id.dana
14398,gp:AOqpTOFltmgvCwjmHxSxud-qON7Bbic8-3XyDEAuVX9...,arinta sasa,https://play-lh.googleusercontent.com/a-/AOh14...,Di update gak tambah bagus malah tambah lelet 👎,5,0,2.2.0,2021-09-10 19:46:37,"Hi Kak, makasih feedbacknya, kalo ada keluhan,...",2021-09-10 11:18:20,newest,id.dana


## Reference
- https://curiousily.com/posts/create-dataset-for-sentiment-analysis-by-scraping-google-play-app-reviews-using-python/